In [1]:
# Load in relevant libraries, and alias where appropriate
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

# Define relevant variables for the ML task
batch_size = 128
num_classes = 10
learning_rate = 0.001
num_epochs = 20

# Device will determine whether to run the training on GPU or CPU.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
#Loading the dataset and preprocessing
train_dataset = torchvision.datasets.MNIST(root = './data',
                                            train = True,
                                            transform = transforms.Compose([
                                                    transforms.Resize((32,32)),
                                                    transforms.ToTensor(),
                                                    transforms.Normalize(mean = (0.1307,), std = (0.3081,))]),
                                            download = True)


test_dataset = torchvision.datasets.MNIST(root = './data',
                                            train = False,
                                            transform = transforms.Compose([
                                                    transforms.Resize((32,32)),
                                                    transforms.ToTensor(),
                                                    transforms.Normalize(mean = (0.1325,), std = (0.3105,))]),
                                            download=True)


train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                            batch_size = batch_size,
                                            shuffle = True)


test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                            batch_size = batch_size,
                                            shuffle = False)

In [ ]:
#Defining the convolutional neural network
class LeNet5(nn.Module):
    def __init__(self, num_classes):
        super(LeNet5, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.fc = nn.Linear(400, 120)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(120, 84)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(84, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.relu(out)
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

In [19]:
#Defining the convolutional neural network
class MyLinearModel(nn.Module):
    def __init__(self, num_classes):
        super(MyLinearModel, self).__init__()
        self.flatten = nn.Flatten()
        self.layer1 = nn.Sequential(
            nn.Linear(32 * 32, 4096),
            nn.ReLU())
        self.layer2 = nn.Sequential(
            nn.Linear(4096, 1024),
            nn.ReLU())
        self.layer3 = nn.Sequential(
            nn.Linear(1024, 256),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Linear(256, 64),
            nn.ReLU())
        self.fc = nn.Linear(64, num_classes)
        
    def forward(self, x):
        # print(x.shape)
        out = self.flatten(x)
        # print(out.shape)
        out = self.layer1(out)
        # print(out.shape)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.fc(out)
        return out

In [20]:
model = MyLinearModel(num_classes).to(device)
    
#Setting the loss function
cost = nn.CrossEntropyLoss()

#Setting the optimizer with the model parameters and learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#this is defined to print how many steps are remaining when training
total_step = len(train_loader)

In [21]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        images = images.to(device)
        labels = labels.to(device)
        
        #Forward pass
        outputs = model(images)
        loss = cost(outputs, labels)
        #Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i+1) % 400 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
.format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/20], Step [400/469], Loss: 0.1466
Epoch [2/20], Step [400/469], Loss: 0.0788
Epoch [3/20], Step [400/469], Loss: 0.0672
Epoch [4/20], Step [400/469], Loss: 0.0981
Epoch [5/20], Step [400/469], Loss: 0.0400
Epoch [6/20], Step [400/469], Loss: 0.0626
Epoch [7/20], Step [400/469], Loss: 0.0394
Epoch [8/20], Step [400/469], Loss: 0.0262
Epoch [9/20], Step [400/469], Loss: 0.0666
Epoch [10/20], Step [400/469], Loss: 0.0257
Epoch [11/20], Step [400/469], Loss: 0.0528
Epoch [12/20], Step [400/469], Loss: 0.0090
Epoch [13/20], Step [400/469], Loss: 0.0011
Epoch [14/20], Step [400/469], Loss: 0.0197
Epoch [15/20], Step [400/469], Loss: 0.0214
Epoch [16/20], Step [400/469], Loss: 0.0123
Epoch [17/20], Step [400/469], Loss: 0.0017
Epoch [18/20], Step [400/469], Loss: 0.0055
Epoch [19/20], Step [400/469], Loss: 0.0023
Epoch [20/20], Step [400/469], Loss: 0.0014


In [22]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
    
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 98.45 %
